In [182]:
import pandas as pd
import numpy as np
## Задача: 
## 1. расчет среднего, мин-макс, медианы, моды для переменных в данных в целом.
## 2. расчет среднего, мин-макс, медианы, моды для переменных в данных в зависимости от уровней переменной Type.
## 3. найти самый популярный объект в выборке; объяснить это. 

# считываем данные с разделителем ';'
df = pd.read_csv('dataset_Facebook.csv', sep = ";")

# просматриваем первые 5 строк
df.head(5)

# посмотрим на типы данных в дата фрейме
df.dtypes

## Если посмотреть описание данных в оригинальной статье
## ('https://www.sciencedirect.com/science/article/abs/pii/S0148296316000813?via%3Dihub'), то
## мы имеем 3 категориальных (номинативных) переменных: Type, Category, Paid. 
## Мы исключим их из расчета показателей для остальных переменных. 
## Для категориальных переменных мы будем использовать моду и расчет частотных таблиц.  

Page total likes                                                         int64
Type                                                                    object
Category                                                                 int64
Post Month                                                               int64
Post Weekday                                                             int64
Post Hour                                                                int64
Paid                                                                   float64
Lifetime Post Total Reach                                                int64
Lifetime Post Total Impressions                                          int64
Lifetime Engaged Users                                                   int64
Lifetime Post Consumers                                                  int64
Lifetime Post Consumptions                                               int64
Lifetime Post Impressions by people who have liked y

In [183]:
## изменим тип данных на category
cat_cols = ['Type', 'Paid', 'Category']
for col in cat_cols:
    df[col] = df[col].astype('category')

In [184]:
from scipy import stats
### получим статистику по переменным. медиана является 50 квантилем распределения. 
summary_stats = df.select_dtypes(exclude = ["category"]).describe()
### Посчитаем моду. 
mode_values = df.select_dtypes(exclude = ["category"]).agg(stats.mode)

In [185]:
### нахождение моды для категориальных переменных "Type", "Category" и "Paid"
df.describe(include=['category'])## наиболее популярны посты с фотографиями; в категории контента наиболее часто встречающийся 
                                  ## элемент выборки - контент, который был классифицирован как action. 
                                  ## Большинство постов из выборки не были оплачены. 

,Type,Category,Paid
count,500,500,499.0
unique,4,3,2.0
top,Photo,1,0.0
freq,426,215,360.0


In [186]:
### частотные таблицы для категориальных переменных
for col in cat_cols:
    print(df[col].value_counts(), end= "\n"*2)

Photo     426
Status     45
Link       22
Video       7
Name: Type, dtype: int64

0.0    360
1.0    139
Name: Paid, dtype: int64

1    215
3    155
2    130
Name: Category, dtype: int64



In [187]:
### статистики в контексте уровней переменной Type.

### среднее, мин-макс, медиана
type_levels_values = df.drop(['Category', 'Paid'], axis = 1).groupby("Type").agg(['min','max','mean','median'])
## мода
types_levels_mode_values = df.drop(['Category', 'Paid'], axis = 1).groupby("Type").agg(stats.mode)

In [188]:
## Наиболее популярный объект в данных.
## Можно по-разному оценить популярность действия на Facebook. В оригинальном исследовании 
## использовалась подсчитанная переменная "Total Interactions" в качестве суммы комментариев + лайков + репостов. 
## Отсортиуем данные и получим необходимое наблюдение. 

df.sort_values(by = ["Total Interactions"], ascending= False)[:1] ## 244 элемент, пост с фото

## Подобный подход также использовался в одном из исследований факторов, влияющих на популярность бренда на фейсбуке, исследователи
## использовали метрику популярности как сумму лайков и комментариев (http://www.sciencedirect.com/science/article/pii/S0263237314000607).

## В качестве альтернативы, мы можем оценить посты через охват аудитории. В таком случае, первое место занимает 244 элемент. 

df.sort_values(by = ["Lifetime Post Total Reach"], ascending= False)[:1] ## 244 элемент

## Помимо этого, можно использовать вовлеченность в качестве индикатора популярности. 
## Вовлеченность влияет на количество показов поста. Уровень вовлеченности отражает количество заинтересованных потребителей, 
## а высокий уровень вовлеченности означает, что все больше людей комментируют, любят, делятся и упоминают о вашем бренде и его контенте.
## На Facebook расчитывается следующим образом: 'Engagement = likes + comments + shares + clicks'. 
## Когда пользователей кликает по сообщению, просматривает фотографию, смотрит видео или, например, нажимает "увидеть больше", то он отмечается как вовлеченный. 
## Ограничение этой оценки в том, что эти клики могут быть бессмысленными (например, человек кликает на индикатор времени в видео, чтобы его мотать).
## Отсортириуем данные. 

df.sort_values(by = ["Lifetime Engaged Users"], ascending= False)[:1] ## 446 элемент, пост с фото

## Мы получили пост 446 пост (фото) с наибольшей вовлеченностью. Однако, если взглянуть на его показатели Total Interactions (всего 228), то вероятно, что
## что мы столкнулись с той ситуацией, когда были выражены "бессмысленные клики". Возможно, это связано с хар-ками самой фотографии:
## превью вынуждало кликать на фотографию. 

## Поэтому на мой взгляд, несмотря на разные подходы к оценки популярности (популярность как совокупность завершенных действий или популярность как потенциальный охват), 
## можно сказать, что объект номер 244 является самым попурялрным в выборке. 



,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
446,96749,Photo,1,3,1,2,1.0,98816,125026,11452,11328,18115,16682,10336,1356,10,197.0,21.0,228
